In [18]:
# Loading data

import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt
import tensorly as tl

# Import the data of a CSV file into a numpy array
cats = genfromtxt('cat.csv', delimiter=',') # 80 cats (one in a column) each by 4096 pixels
dogs = genfromtxt('dog.csv', delimiter=',') # 80 dogs (one in a column) each by 4096 pixels

# Combine two arrays (cats and dogs) into one tensor of size 64x64x160 (pets)
pets = np.empty([64, 64, 160])
for m in range(80):
    pets[:,:,m] = np.transpose(np.reshape(cats[:,m], (64,64)))
    pets[:,:,m+80] = np.transpose(np.reshape(dogs[:,m], (64,64)))

In [19]:
# Converting data using edge detection
from skimage.filters import sobel
edge_pets = np.empty([64, 64, 160])
for m in range(160):
    edge_pets[:,:,m] = sobel(pets[:,:,m])

In [20]:
# Converting data using thresholding
import cv2
thresh_pets = np.empty([64, 64, 160])
for m in range(160):
    retval,img = cv2.threshold(pets[:,:,m], 135, 255, cv2.THRESH_BINARY)
    thresh_pets[:,:,m] = img

In [21]:
# Feature extraction using tensor decomposition
from tensorly.decomposition import parafac
factors = parafac(edge_pets, rank = 3) # Calculating the CP decomposition
features = factors[2] # Features matrix of size 160 by rank
print([f.shape for f in factors])

[(64L, 3L), (64L, 3L), (160L, 3L)]


In [22]:
def ClassifyData(data):

    labels = np.concatenate((np.ones(80), [2] * 80), axis=0)
    # data and labels for training
    train_data = np.concatenate((data[:50, :], data[80:130,:]), axis=0)
    train_labels = np.concatenate((labels[:50], labels[80:130]), axis=0)
    # data and labels for testing
    test_data = np.concatenate((data[50:80, :], data[130:,:]), axis=0)
    test_labels = np.concatenate((labels[50:80], labels[130:]), axis=0)

    # Apply ML algorithms

    scores = []

    # Linear Discriminant Analysis (LDA)
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
    lda = LDA()
    clf = lda.fit(train_data, train_labels)
    scores.append(clf.score(test_data, test_labels))
    
    # Support Vector Machine (SVM)
    from sklearn import svm
    clf = svm.SVC(kernel='linear')
    clf.fit(train_data, train_labels)
    scores.append(clf.score(test_data, test_labels))

    # Gaussian Naive Bayes (GNB)
    from sklearn.naive_bayes import GaussianNB
    clf = GaussianNB()
    clf.fit(train_data, train_labels)
    scores.append(clf.score(test_data, test_labels))

    return [i * 100 for i in scores]

In [23]:
print(ClassifyData(features))

[80.0, 78.333333333333329, 75.0]
